In [ ]:
import sqlite3

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas import Series

from utils.dataset_utils import PoliticalLeaningDataset
from utils.existing_models.politicalness import PoliticalDebateLarge

Reading the raw dataset.

In [ ]:
connection = sqlite3.connect("../raw/commoncrawl_news_articles/articles.db")

df = pd.read_sql(
    "SELECT content_preprocessed, au.outlet_name FROM article_contents ac INNER JOIN article_urls au on ac.uuid = au.uuid WHERE language = 'en'",
    connection
)

outlets_df = pd.read_json(open("../raw/commoncrawl_news_articles/outlets.json"))
df = df.merge(outlets_df, how="outer", left_on="outlet_name", right_on="name")
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["outlet_name", "name", "tld", "filter", "allsides_name"])

Renaming columns.

In [ ]:
df = df.rename(columns={"content_preprocessed": "body", "allsides_rating": "leaning"})

Categorizing the leaning label column.

In [ ]:
df["leaning"].unique()

In [ ]:
df["leaning"] = df["leaning"].replace({"lean left": "left", "lean right": "right"})
df["leaning"] = df["leaning"].astype("category")

Adding the politicalness label column.

In [ ]:
df["politicalness"] = pd.Categorical(["political"] * len(df))

Stripping the bodies.

In [ ]:
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Replacing `<SENT_END>`s by spaces.

In [ ]:
df["body"] = df["body"].str.replace("<SENT_END>", " ")

Inspecting the body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

After the inspection, rows with bodies shorter than 50 words seem to contain no political value. Removing them.

In [ ]:
body_word_count_lower_bound = 50
df = df[df["body_word_count"] >= body_word_count_lower_bound]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

Removing rows with non-political bodies.

In [ ]:
NON_POLITICAL_LABEL_SCORE_MIN = 0.99

model = PoliticalDebateLarge()


def is_political(bodies: Series) -> Series:
    results = model.predict_batch_with_score(list(bodies))
    return Series(map(
        lambda result: not (result[0] == 0 and result[1] >= NON_POLITICAL_LABEL_SCORE_MIN),
        results,
    ), index=bodies.index)


df = df.loc[::100000]
df = df[is_political(df["body"])]

Downsampling the dataset.

In [ ]:
SAMPLE_SIZE = 100_000

dataset = PoliticalLeaningDataset("commoncrawl_news_articles", df)
df = dataset.take_even_class_distribution_sample(SAMPLE_SIZE).dataframe

Inspecting the body word count mean.

In [ ]:
df["body_word_count"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

The body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    df_leaning = df[df["leaning"] == leaning]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_leaning["body_length"].iloc[::100], df_leaning["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/commoncrawl_news_articles.parquet")